In [6]:
import pandas as pd
import pickle as pkl
import re
from collections import Counter
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords 

import stanza
# stanza.download('en')
nlp = stanza.Pipeline(lang='en', processors='tokenize,mwt,pos,lemma')

def count_freq(preds, gts):
    preds_len = [len(x.split(' ')) for x in preds]
    gts_len = [len(x.split(' ')) for x in gts]
    print(Counter(preds_len))
    print(Counter(gts_len))

def evaluation_metrics(pred, gt):
    print("Prediction: ", len(set(pred)))
    print("GT: ",len(set(gt)))
    TP = len(set(pred) & set(gt)) 
    FP = len(set(pred)-set(gt))
    FN = len(set(gt)-set(pred))
    print(TP,FP,FN)
    precision = round((TP/(TP+FP))*100, 2)
    recall = round((TP/(TP+FN))*100,2)
    f1_score = round((2 * precision * recall) / (precision + recall),2)
    return precision, recall, f1_score 

def lemma(li):
    new_list = []
    for t in li:
        doc = nlp(str(t))
        doc1 = ' '.join([word.lemma for sent in doc.sentences for word in sent.words])
        doc1 = re.sub('-',' ',doc1)
        doc1 = re.sub(' +', ' ',doc1)
        new_list.append(doc1)
    new_list = [s for s in new_list if len(s) >= 2]
    return new_list

def get_term_(predictions):
    all_term = []
    for sentence in predictions:
        tokens = []
        labels = []
        for d in sentence:
            tokens.extend(d.keys())
            labels.extend(d.values())

        for i, label in enumerate(labels):
            if labels[i] == 'I' and (i == 0 or labels[i - 1] == 'O'):
                labels[i] = 'O'

        terms = []
        term = []
        for token, label in zip(tokens, labels):
            if label == 'B':
                #Lưu vị trí B
                b_pos = i
                term = [token]
            elif label == 'I':
                term.append(token)
            elif len(term) > 0:
                terms.append(' '.join(term))
                term = []
        if len(term) > 0:
            terms.append(' '.join(term))
            # Check b_pos = 0 không
        all_term.append(terms)
    
    final_terms = []
    for i in all_term:
        final_terms.extend(i)

    final_terms = [x.lower().strip() for x in final_terms]
    return final_terms  

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/hanhtran/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
2021-06-03 10:40:38 WARNING: Can not find mwt: default from official model list. Ignoring it.
2021-06-03 10:40:38 INFO: Loading these models for language: en (English):
| Processor | Package  |
------------------------
| tokenize  | combined |
| pos       | combined |
| lemma     | combined |

2021-06-03 10:40:38 INFO: Use device: cpu
2021-06-03 10:40:38 INFO: Loading: tokenize
2021-06-03 10:40:38 INFO: Loading: pos
2021-06-03 10:40:38 INFO: Loading: lemma
2021-06-03 10:40:38 INFO: Done loading processors!


In [7]:
def get_term(predictions):
    all_term = []
    for sentence in predictions:
        tokens = []
        labels = []
        for d in sentence:
            tokens.extend(d.keys())
            labels.extend(d.values())

        for i, label in enumerate(labels):
            if labels[i] == 'I' and (i == 0 or labels[i - 1] == 'O'):
                labels[i] = 'O'

        terms = []
        term = []
        for i, (token, label) in enumerate(zip(tokens, labels)):
            if label == 'B': 
                #Lưu vị trí B
                b_pos = i
                term = [token]
            elif label == 'I':
                term.append(token)
            elif len(term) > 0:
                terms.append(' '.join(term))
                # Check b_pos = 0 không
                if b_pos != 0:
                    if (tokens[b_pos - 1] != '') and (tokens[b_pos - 1] != ' '):
                        if len(nlp(str(tokens[b_pos - 1])).sentences) > 0:
                            b_word = nlp(str(tokens[b_pos - 1])).sentences[0].words[0]
                            if len(nlp(str(tokens[b_pos])).sentences) > 0:
                                c_word = nlp(str(tokens[b_pos])).sentences[0].words[0]
                                if (c_word.upos == 'NOUN') and (c_word.text != 'None') and (b_word.text != 'None') and ((b_word.upos == 'NOUN') or (b_word.upos == 'ADJ')):
                                    terms.append(' '.join([b_word.text] + term))
                    if (tokens[i] != '') and (tokens[i] != ' '):
                        if len(nlp(str(tokens[i])).sentences) > 0:
                            a_word = nlp(str(tokens[i])).sentences[0].words[0]
                            if (a_word.text != 'None') and (a_word.upos == 'NOUN'):
                                terms.append(' '.join(term + [a_word.text]))
                    
                    #ADJ NOUN NOUN
                    if (tokens[i] != '') and (tokens[i] != ' ') and (tokens[b_pos - 1] != '') and (tokens[b_pos - 1] != ' '):
                        if len(nlp(str(tokens[b_pos - 1])).sentences) > 0 and len(nlp(str(tokens[i])).sentences) > 0:
                            a_word = nlp(str(tokens[i])).sentences[0].words[0]
                            b_word = nlp(str(tokens[b_pos - 1])).sentences[0].words[0]
                            if len(nlp(str(tokens[b_pos])).sentences) > 0:
                                c_word = nlp(str(tokens[b_pos])).sentences[0].words[0]
                                # Check vị trí b_pos - 1: terms.append()
                                if (c_word.text != 'None') and (c_word.upos == 'NOUN') and (b_word.text != 'None') and ((b_word.upos == 'ADJ') and (a_word.upos == 'NOUN')):
                                    terms.append(' '.join([b_word.text] + term + [a_word.text]))
#                                     print(terms)
                    #ADJ ADJ NOUN              
                    if (tokens[b_pos - 2] != '') and (tokens[b_pos - 2] != ' ') and (tokens[b_pos - 1] != '') and (tokens[b_pos - 1] != ' '):
                        if (len(nlp(str(tokens[b_pos - 2])).sentences) > 0 and len(nlp(str(tokens[b_pos - 1])).sentences) > 0 and len(nlp(str(tokens[b_pos])).sentences) > 0):
                            b1_word = nlp(str(tokens[b_pos - 2])).sentences[0].words[0] 
                            b_word = nlp(str(tokens[b_pos - 1])).sentences[0].words[0]
                            c_word = nlp(str(tokens[b_pos])).sentences[0].words[0]
                            # Check vị trí b_pos - 1: terms.append()
                            if (c_word.upos == 'NOUN') and (c_word.text != 'None') and (b_word.text != 'None') and (b1_word.text != 'None') and ((b_word.upos == 'ADJ') and (b1_word.upos == 'ADJ')):
                                terms.append(' '.join([b1_word.text] +[b_word.text] + term))
#                                 print(terms)
                    #NOUN ADJ NOUN
                    if (tokens[b_pos - 2] != '') and (tokens[b_pos - 2] != ' ') and (tokens[b_pos - 1] != '') and (tokens[b_pos - 1] != ' '):
                        if (len(nlp(str(tokens[b_pos - 2])).sentences) > 0 and len(nlp(str(tokens[b_pos - 1])).sentences) > 0 and len(nlp(str(tokens[b_pos])).sentences) > 0):
                            b1_word = nlp(str(tokens[b_pos - 2])).sentences[0].words[0] 
                            b_word = nlp(str(tokens[b_pos - 1])).sentences[0].words[0]
                            c_word = nlp(str(tokens[b_pos])).sentences[0].words[0]
                            # Check vị trí b_pos - 1: terms.append()
                            if (c_word.upos == 'NOUN') and (c_word.text != 'None') and (b_word.text != 'None') and (b1_word.text != 'None') and ((b_word.upos == 'ADJ') and (b1_word.upos == 'NOUN')):
                                terms.append(' '.join([b1_word.text] +[b_word.text] + term))
#                                 print(terms)
                                
                # NOUN NOUN NOUN
                if b_pos != 0 and i + 1 < len(tokens):
                    if (tokens[i] != '') and (tokens[i] != ' ') and (tokens[i+1] != '') and (tokens[i+1] != ' '):
                        if (len(nlp(str(tokens[i])).sentences) > 0 and len(nlp(str(tokens[i+1])).sentences) > 0 and len(nlp(str(tokens[b_pos])).sentences) > 0):
                            a1_word = nlp(str(tokens[i+1])).sentences[0].words[0] 
                            a_word = nlp(str(tokens[i])).sentences[0].words[0]
                            c_word = nlp(str(tokens[b_pos])).sentences[0].words[0]
                            if (c_word.text != 'None') and (c_word.upos == 'NOUN') and (a_word.text != 'None') and (a1_word.text != 'None') and ((a_word.upos == 'NOUN') and (a1_word.upos == 'NOUN')):
                                terms.append(' '.join( term + [a_word.text] +[a1_word.text]))
#                                 print(terms)
                if b_pos != 0:
                    # RULE 2
                    if (tokens[b_pos-2] != '') and (tokens[b_pos -2] != ' ') and (tokens[b_pos - 1] != '') and (tokens[b_pos - 1] != ' '):
                        if len(nlp(str(tokens[b_pos - 1])).sentences) > 0 and len(nlp(str(tokens[b_pos-2])).sentences) > 0:
                            b_word = nlp(str(tokens[b_pos - 1])).sentences[0].words[0]
                            b1_word = nlp(str(tokens[b_pos - 2])).sentences[0].words[0]
                            # Check vị trí b_pos - 1: terms.append()
                            if  (b_word.text == '-') and (b1_word.text != 'None'):
                                terms.append(' '.join([b1_word.text] + [b_word.text] + term))
                                print(terms)
                    
                term = []
        if len(term) > 0:
            terms.append(' '.join(term))
            # check b_pos - 1
        all_term.append(terms)
    
    final_terms = []
    for i in all_term:
        final_terms.extend(i)

    final_terms = [x.lower().strip() for x in final_terms]
    return final_terms    

In [9]:
groundtruth = pd.read_csv('/Users/hanhtran/Documents/terminology-extraction/ACTER/fr/htfl/annotations/htfl_fr_terms_nes.ann', sep='	', engine='python',header=None)
gt = list(groundtruth[0]) 
def lemma(doc):
    lemma_word = nlp(str(doc))
    lemma_word = ' '.join([w.lemma for sent in lemma_word.sentences for w in sent.words])
    lemma_word = re.sub(' -','-',lemma_word)
    lemma_word = re.sub('- ','-',lemma_word)
    lemma_word = re.sub(' \)', ' ',lemma_word)
    lemma_word = re.sub('\( ', ' ',lemma_word)
    lemma_word = re.sub(' +', ' ',lemma_word)
    return lemma_word
        
pkl_file = open('./fr/nes_distilbert_cased.pkl', 'rb') 
predictions = pkl.load(pkl_file)

preds =  get_term_(predictions)
print(evaluation_metrics(preds, gt))
stop_words = set(stopwords.words('english'))
pred_terms =  set(preds) - set(stop_words)
pred_terms = [x for x in pred_terms if len(x)>1]
pred_terms = [x.lower().strip() for x in pred_terms]
pred_terms = [re.sub(' -','-', x) for x in pred_terms]
pred_terms = [re.sub('- ','-', x) for x in pred_terms]
pred_terms = [re.sub('\(','', x) for x in pred_terms]
pred_terms = [re.sub('\/','', x) for x in pred_terms]
print(evaluation_metrics(pred_terms, gt))

gt = list(set([lemma(x) for x in gt]))
print(evaluation_metrics(pred_terms, gt))

In [14]:
groundtruth = pd.read_csv('/Users/hanhtran/Documents/terminology-extraction/ACTER/en/htfl/annotations/htfl_en_terms.ann', sep='	', engine='python',header=None)
gt = list(groundtruth[0]) 

# pkl_file = open('./en/ann_weighted_xlnet.pkl', 'rb') 
pkl_file = open('ann_xlnet_full.pkl', 'rb') 
predictions = pkl.load(pkl_file)

preds =  get_term_(predictions)
print(evaluation_metrics(preds, gt))
stop_words = set(stopwords.words('english'))
pred_terms =  set(preds) - set(stop_words)
pred_terms = [x for x in pred_terms if len(x)>1]
pred_terms = [x.lower().strip() for x in pred_terms]
pred_terms = [re.sub(' -','-', x) for x in pred_terms]
pred_terms = [re.sub('- ','-', x) for x in pred_terms]
pred_terms = [re.sub('\(','', x) for x in pred_terms]
pred_terms = [re.sub('\/','', x) for x in pred_terms]
print(evaluation_metrics(pred_terms, gt))

Prediction:  1197
GT:  2361
456 741 1905
(38.1, 19.31, 25.63)
Prediction:  1181
GT:  2361
454 727 1907
(38.44, 19.23, 25.64)


In [7]:
preds =  get_term(predictions)
print(evaluation_metrics(preds, gt))
stop_words = set(stopwords.words('english'))
pred_terms =  set(preds) - set(stop_words)
pred_terms = [x for x in pred_terms if len(x)>1]
pred_terms = [x.lower().strip() for x in pred_terms]
pred_terms = [re.sub(' -','-', x) for x in pred_terms]
pred_terms = [re.sub('- ','-', x) for x in pred_terms]
pred_terms = [re.sub('\(','', x) for x in pred_terms]
pred_terms = [re.sub('\/','', x) for x in pred_terms]
print(evaluation_metrics(pred_terms, gt))

['Aldosterone', 'neurohormone', 'heart failure', 'systolic dysfunction', 'aldosterone antagonist', 'heart failure', 'Aldactone', 'Aldactone Evaluation', 'Aldactone Evaluation Study', 'chronic', 'heart failure', 'advanced heart failure', 'heart failure patients', 'advanced heart failure patients', 'chronic advanced heart failure', 'aldosterone antagonist', 'spironolactone', 'placebo', 'beta blockade', 'myocardial', 'myocardial infarction', 'Efficacy', 'failure Efficacy', 'SUrvival Study', 'EPHESUS', 'post', 'myocardial', 'myocardial infarction', 'post - myocardial']
['Aldosterone', 'neurohormone', 'heart failure', 'systolic dysfunction', 'aldosterone antagonist', 'heart failure', 'Aldactone', 'Aldactone Evaluation', 'Aldactone Evaluation Study', 'chronic', 'heart failure', 'advanced heart failure', 'heart failure patients', 'advanced heart failure patients', 'chronic advanced heart failure', 'aldosterone antagonist', 'spironolactone', 'placebo', 'beta blockade', 'myocardial', 'myocardia

['ambulatory', 'ambulatory patients', 'heart failure', 'renin', 'angiotensin system', 'renin - angiotensin system', 'blockers', 'β - blockers', 'readmissions', 'blocker', 'blocker uptitrations', 'β - blocker']
['ambulatory', 'ambulatory patients', 'heart failure', 'renin', 'angiotensin system', 'renin - angiotensin system', 'blockers', 'β - blockers', 'readmissions', 'blocker', 'blocker uptitrations', 'β - blocker', 'ventricular', 'ventricular ejection', 'left ventricular', 'left ventricular ejection', 'blocker', 'blocker dose', 'blocker dose changes', 'β - blocker']
['ambulatory', 'ambulatory patients', 'heart failure', 'renin', 'angiotensin system', 'renin - angiotensin system', 'blockers', 'β - blockers', 'readmissions', 'blocker', 'blocker uptitrations', 'β - blocker', 'ventricular', 'ventricular ejection', 'left ventricular', 'left ventricular ejection', 'blocker', 'blocker dose', 'blocker dose changes', 'β - blocker', 'blocker', 'blocker dose', 'blocker dose increase', 'β - block

['HFrEF', 'exercise protocol', 'gas', 'respiratory gas', 'gas analysis', 'respiratory gas analysis', 'beta', 'blocker therapy', 'beta - blocker therapy', 'resting', 'resting HR', 'blocker dose', 'Beta - blocker dose']
['HFrEF', 'exercise performance', 'HR', 'beta', 'blocker therapy', 'beta - blocker therapy']
['HFrEF', 'exercise performance', 'HR', 'beta', 'blocker therapy', 'beta - blocker therapy', 'HR reduction', 'little HR reduction', 'beta', 'blocker therapy', 'beta - blocker therapy']
['AAs', 'HF', 'hyperkalemia', 'less hyperkalemia', 'hypokalemia', 'more hypokalemia', 'spironolactone', 'AAs', 'non - AAs']
['transplantation', 'cell transplantation', 'heart failure', 'congestive heart failure', 'developing congestive heart failure', 'CHF', 'anthracyclines', 'CHF', 'anthracycline metabolism', 'iron homeostasis', 'anti', 'oxidant defence', 'anti - oxidant defence']
['transplantation', 'cell transplantation', 'heart failure', 'congestive heart failure', 'developing congestive heart f

['plasma', 'HF', 'HF patients', 'immunoreactive', 'proBNP', 'NT - proBNP', 'terminally', 'C - terminally', 'the', 'proBNP molecule', 'NT - proBNP molecule', 'proBNP', 'NT - proBNP', 'terminal regions', 'proBNP assay', 'NT - proBNP assay']
['the', 'the changes', 'platelet', 'activating factor', 'platelet - activating factor']
['the', 'the changes', 'platelet', 'activating factor', 'platelet - activating factor', 'metabolic enzymes', 'heart failure', 'HF', 'The', 'biosynthetic enzymes', 'acetyltransferase', 'dithiothreitol', 'choline', 'CDP choline', 'cholinephosphotransferase', 'catabolic isoenzymes', 'acetylhydrolase', 'PAF - acetylhydrolase']
['longitudinal', 'longitudinal strain', 'ventricle', 'echocardiography', 'baseline echocardiography', 'CRT', 'echocardiograms', 'Regression', 'Regression analysis', 'reverse', 'reverse remodeling', 'the', 'systolic volume', 'end - systolic volume']
['longitudinal', 'longitudinal strain', 'ventricle', 'echocardiography', 'baseline echocardiography

['HF', 'heart failure', 'T2D', 'diabetes', 'SkM', 'muscle', 'skeletal muscle', 'We', 'epicatechin', 'cocoa', 'mitochondrial structure', 'Taub', 'Ramirez', 'Sanchez', 'Ramirez - Sanchez']
['ejection', 'ventricular ejection', 'ejection fraction', 'ventricular ejection fraction', 'idiopathic', 'cardiomyopathy', 'dilated cardiomyopathy', 'idiopathic dilated cardiomyopathy', 'angiography', 'coronary angiography', 'heart failure', 'heart failure treatment', 'heart failure treatment eras', 'angiotensin', 'blockers', 'receptors blockers', 'blockers', 'β - blockers']
['breathing control', 'study breathing control', 'cardiac arrest', 'humans', 'fibrillation', 'ventricular fibrillation', 'pulseless ventricular fibrillation', 'cardioverter', 'defibrillator device', 'cardioverter - defibrillator device']
['the', 'cardiovascular diseases', 'heart failure', 'HF', 'We', 'troponin', 'cardiac troponin', 'cTnT', 'terminal', 'N - terminal']
['the', 'cardiovascular diseases', 'heart failure', 'HF', 'We', '

['Heart failure', 'HF', 'HF', 'ejection fraction', 'HF', 'ejection fraction', 'HFpEF', 'renin', 'angiotensin', 'renin - angiotensin', 'aldosterone system', 'angiotensin - aldosterone system']
['Heart failure', 'HF', 'HF', 'ejection fraction', 'HF', 'ejection fraction', 'HFpEF', 'renin', 'angiotensin', 'renin - angiotensin', 'aldosterone system', 'angiotensin - aldosterone system', 'RAAS', 'HFrEF', 'angiotensin', 'converting enzyme inhibitors', 'angiotensin - converting enzyme inhibitors']
['retest', 'retest reliability', 'test - retest']
['retest', 'retest reliability', 'test - retest']
['Cross', 'sectional', 'Cross - sectional']
['concentric', 'concentric remodeling', 'concentric', 'concentric hypertrophy', 'ventricular', 'heart failure', 'ejection fraction', 'eccentric', 'eccentric hypertrophy', 'wall thickness', 'relative wall thickness', 'echocardiographic', 'hemodynamic', 'systolic', 'lower systolic', 'systolic blood', 'lower systolic blood', 'systolic blood pressure', 'renal impa

In [31]:
# groundtruth = pd.read_csv('/Users/hanhtran/Documents/terminology-extraction/ACTER/nl/htfl/annotations/htfl_nl_terms.ann', sep='	', engine='python',header=None)
# gt = list(groundtruth[0]) 

# pkl_file = open('./nl/nes_weighted_distilbert_multilingual_cased.pkl', 'rb') 
# predictions = pkl.load(pkl_file)

# preds =  get_term_(predictions)
# print(evaluation_metrics(preds, gt))
# stop_words = set(stopwords.words('english'))
# pred_terms =  set(preds) - set(stop_words)
# pred_terms = [x for x in pred_terms if len(x)>1]
# pred_terms = [x.lower().strip() for x in pred_terms]
# pred_terms = [re.sub(' -','-', x) for x in pred_terms]
# pred_terms = [re.sub('- ','-', x) for x in pred_terms]
# pred_terms = [re.sub('\(','', x) for x in pred_terms]
# pred_terms = [re.sub('\/','', x) for x in pred_terms]
# evaluation_metrics(pred_terms, gt)

In [16]:
pred_terms

['ladderpatroon',
 'gedragsmatige',
 'sinusritme',
 'boezemfibrilleren',
 'cyp2c9',
 'spier-',
 'myocardfibrose',
 'ascitesvocht',
 'nierfunctie-verslechtering',
 'nabelasting',
 'vochtrestrictie',
 'geleidingssnelheid',
 'hartslag',
 'bradycardie',
 'voorwaarts',
 'arts-assistenten',
 'mmhg',
 'post-hoc',
 'palpabel',
 'serumcholesterol-',
 'remmen',
 'ischemische',
 'h2o.',
 'verzamelden',
 'dialysetherapie',
 'ramus',
 'farmacogenetica',
 'angiotensine-receptor-blokkers',
 'onregelmatig',
 'winst',
 'mitralis-aortaklepinsufficiëntie',
 'vaatlijden',
 'gemeenschappelijk',
 'supraventriculaire',
 'natriumsuppletie',
 'dotterbehandeling',
 'cumarinen',
 'thuisinfusietherapie',
 'calciumantagonisten',
 'voorkómen',
 'telefoon',
 'basistherapie',
 'hartfalenverpleegkundige',
 'klasse-iv-patiënt',
 'antracyclinederivaten',
 'longcrepitaties',
 'cardiomyopathie',
 'alcoholinname',
 'glitazones',
 'linkerventrikeldruk',
 'receptorstructuur',
 'cross-overonderzoek',
 'trager',
 'ziekenhuisop

In [ ]:
# preds_lem = lemma(pred_terms)
# gts_lem = lemma(gt)

# print('After lemmatization')
# print(len(set(preds_lem)))
# print(len(set((gts_lem))))
# print(evaluation_metrics(list(set(preds_lem)), list(set(gts_lem))))

In [ ]:
# groundtruth = pd.read_csv('/Users/hanhtran/Documents/terminology-extraction/ACTER/en/htfl/annotations/htfl_en_terms.ann', sep='	', engine='python',header=None)
# gt = list(groundtruth[0]) 

# pkl_file = open('../rules/final_preds_sklearn.pkl', 'rb') #
# predictions = pkl.load(pkl_file)
# preds =  get_term_(predictions)
# print(evaluation_metrics(preds, gt))
# stop_words = set(stopwords.words('english'))
# pred_terms =  set(preds) - set(stop_words)
# pred_terms = [x for x in pred_terms if len(x)>1]
# pred_terms = [x.lower().strip() for x in pred_terms]
# pred_terms = [re.sub(' -','-', x) for x in pred_terms]
# pred_terms = [re.sub('- ','-', x) for x in pred_terms]
# pred_terms = [re.sub('\(','', x) for x in pred_terms]
# pred_terms = [re.sub('\/','', x) for x in pred_terms]
# evaluation_metrics(pred_terms, gt)

In [ ]:
col = ['a','b','c','d']
for num, alpha in enumerate('abcd'):
    print(num)
    print(alpha)

In [ ]:
for num, alpha in enumerate(col):
    print(num)
    print(alpha)

In [ ]:
# def sephist(col):
#     yes = df[df['group'] == 'yes'][col]
#     no = df[df['group'] == 'no'][col]
#     return yes, no

# for num, alpha in enumerate('abcd'):
#     plt.subplot(2, 2, num)
#     plt.hist(sephist(alpha)[0], bins=25, alpha=0.5, label='yes', color='b')
#     plt.hist(sephist(alpha)[1], bins=25, alpha=0.5, label='no', color='r')
#     plt.legend(loc='upper right')
#     plt.title(alpha)
# plt.tight_layout(pad=0.4, w_pad=0.5, h_pad=1.0)

# #https://stackoverflow.com/questions/45883598/pandas-histogram-df-hist-group-by

In [13]:
nlp = stanza.Pipeline('en', processors='tokenize', use_gpu=True, pos_batch_size=3000) # Build the pipeline, specify part-of-speech processor's batch size
doc = nlp("Barack Obama was born in Hawaii.") # Run the pipeline on the input text
print(doc) # Look at the result

2021-06-04 10:08:36 INFO: Loading these models for language: en (English):
| Processor | Package  |
------------------------
| tokenize  | combined |

2021-06-04 10:08:36 INFO: Use device: cpu
2021-06-04 10:08:36 INFO: Loading: tokenize
2021-06-04 10:08:36 INFO: Done loading processors!


[
  [
    {
      "id": 1,
      "text": "Barack",
      "misc": "start_char=0|end_char=6"
    },
    {
      "id": 2,
      "text": "Obama",
      "misc": "start_char=7|end_char=12"
    },
    {
      "id": 3,
      "text": "was",
      "misc": "start_char=13|end_char=16"
    },
    {
      "id": 4,
      "text": "born",
      "misc": "start_char=17|end_char=21"
    },
    {
      "id": 5,
      "text": "in",
      "misc": "start_char=22|end_char=24"
    },
    {
      "id": 6,
      "text": "Hawaii",
      "misc": "start_char=25|end_char=31"
    },
    {
      "id": 7,
      "text": ".",
      "misc": "start_char=31|end_char=32"
    }
  ]
]
